<a href="https://colab.research.google.com/github/NaveenMantri26/TEAM-133-DATA_DETECTIVES/blob/main/AI_Personalized_Learning_Mentor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip install transformers accelerate sentencepiece gradio

In [14]:
import random
import time
from typing import List, Dict, Any

import gradio as gr
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

In [15]:
MODEL_NAME = "google/flan-t5-base"

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME).to(device)

def generate_text(prompt: str, max_new_tokens: int = 256) -> str:
    if not prompt.strip():
        return "Please provide a valid input."
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        top_p=0.9,
        temperature=0.7
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

Using device: cpu


In [16]:
# In-memory "database"
students: Dict[str, Dict[str, Any]] = {}

TOPICS = [
    "Physics - Newton's Laws",
    "Mathematics - Quadratic Equations",
    "Biology - Cell Structure",
    "Computer Science - Arrays",
    "Chemistry - Acids and Bases",
]

DIFFICULTY_LEVELS = ["easy", "medium", "hard"]

def init_student(student_id: str, name: str, level: str = "beginner") -> Dict[str, Any]:
    if student_id not in students:
        students[student_id] = {
            "name": name,
            "level": level,
            "current_topic": None,
            "history": [],  # list of dicts: {topic, question, correct, difficulty}
            "scores": {},   # topic -> {"correct": x, "total": y}
            "last_recommendations": [],
        }
    return students[student_id]

In [17]:
def get_topic_mastery(student: Dict[str, Any]) -> Dict[str, float]:
    mastery = {}
    for topic, stats in student["scores"].items():
        if stats["total"] > 0:
            mastery[topic] = stats["correct"] / stats["total"]
        else:
            mastery[topic] = 0.0
    return mastery

def recommend_next_topics(student: Dict[str, Any], top_k: int = 3) -> List[str]:
    mastery = get_topic_mastery(student)
    # Ensure all topics exist in mastery
    for t in TOPICS:
        if t not in mastery:
            mastery[t] = 0.0
    # Sort by lowest mastery (focus on weak areas)
    sorted_topics = sorted(mastery.items(), key=lambda x: x[1])
    recs = [t[0] for t in sorted_topics[:top_k]]
    student["last_recommendations"] = recs
    return recs

def personalize_learning_path(student_id: str, name: str, level: str) -> str:
    student = init_student(student_id, name, level)
    recs = recommend_next_topics(student)

    path_description = f"""Personalized Learning Path for {student['name']} ({student['level']} level):

1. Start with: {recs[0]}
2. Then practice: {recs[1]}
3. Finally, strengthen: {recs[2]}

The system will adapt difficulty based on your answers and update this path dynamically.
"""
    return path_description

In [18]:
def generate_mcq(topic: str, difficulty: str = "easy") -> Dict[str, Any]:
    prompt = f"""
Generate ONE multiple-choice question on the topic: {topic}.
Difficulty: {difficulty}.
Return in this precise format:
QUESTION: <question text>
A) <option A>
B) <option B>
C) <option C>
D) <option D>
ANSWER: <correct option letter>
EXPLANATION: <short explanation>
"""
    raw = generate_text(prompt, max_new_tokens=256)
    question = ""
    options = {}
    answer = ""
    explanation = ""

    for line in raw.split("\n"):
        line = line.strip()
        if line.startswith("QUESTION:"):
            question = line.replace("QUESTION:", "").strip()
        elif line.startswith("A)"):
            options["A"] = line[2:].strip()
        elif line.startswith("B)"):
            options["B"] = line[2:].strip()
        elif line.startswith("C)"):
            options["C"] = line[2:].strip()
        elif line.startswith("D)"):
            options["D"] = line[2:].strip()
        elif line.startswith("ANSWER:"):
            answer = line.replace("ANSWER:", "").strip().upper()[:1]
        elif line.startswith("EXPLANATION:"):
            explanation = line.replace("EXPLANATION:", "").strip()

    if answer not in ["A", "B", "C", "D"]:
        # Fallback if parsing fails
        answer = random.choice(["A", "B", "C", "D"])

    return {
        "topic": topic,
        "difficulty": difficulty,
        "question": question,
        "options": options,
        "answer": answer,
        "explanation": explanation,
    }

In [19]:
def adjust_difficulty(student: Dict[str, Any], last_correct: bool, current_difficulty: str) -> str:
    idx = DIFFICULTY_LEVELS.index(current_difficulty)
    if last_correct and idx < len(DIFFICULTY_LEVELS) - 1:
        idx += 1  # move to harder
    elif not last_correct and idx > 0:
        idx -= 1  # move to easier
    return DIFFICULTY_LEVELS[idx]

In [20]:
def update_student_progress(student_id: str, qa_record: Dict[str, Any]) -> None:
    student = students[student_id]
    topic = qa_record["topic"]
    correct = qa_record["correct"]
    difficulty = qa_record["difficulty"]

    student["history"].append(qa_record)

    if topic not in student["scores"]:
        student["scores"][topic] = {"correct": 0, "total": 0}

    student["scores"][topic]["total"] += 1
    if correct:
        student["scores"][topic]["correct"] += 1

def format_progress_report(student_id: str) -> str:
    student = students.get(student_id)
    if not student:
        return "No data available for this student yet."

    mastery = get_topic_mastery(student)
    lines = [
        f"Progress Dashboard for {student['name']} ({student['level']} level):",
        "-" * 50,
    ]

    for topic in TOPICS:
        m = mastery.get(topic, 0.0)
        stats = student["scores"].get(topic, {"correct": 0, "total": 0})
        lines.append(
            f"{topic}: {stats['correct']}/{stats['total']} correct "
            f"({round(m * 100)}% mastery)"
        )

    lines.append("\nRecent Activity:")
    for entry in student["history"][-5:]:
        status = "✅ Correct" if entry["correct"] else "❌ Wrong"
        lines.append(
            f"- Topic: {entry['topic']} | Diff: {entry['difficulty']} | {status}"
        )

    lines.append("\nRecommended Next Topics:")
    if student["last_recommendations"]:
        for i, t in enumerate(student["last_recommendations"], 1):
            lines.append(f"{i}. {t}")
    else:
        lines.append("Recommendations will appear after you attempt some questions.")

    return "\n".join(lines)

In [21]:
def explain_topic(topic: str, level: str = "beginner") -> str:
    prompt = f"""
Explain the topic "{topic}" to a {level} learner.
Use simple language, short paragraphs, and include one real-world example.
"""
    return generate_text(prompt, max_new_tokens=256)

def chatbot_mentor(student_id: str, message: str) -> str:
    student = students.get(student_id)
    name = student["name"] if student else "Student"

    prompt = f"""
You are an AI mentor helping a student named {name}.
Student profile: {student}.
Respond to the student's message with:
- Encouraging tone
- Clear explanation
- If needed, a follow-up question to check understanding

Student message: {message}
Mentor reply:
"""
    return generate_text(prompt, max_new_tokens=256)

In [22]:
# We'll store current MCQ per student in a session dict
current_mcq: Dict[str, Dict[str, Any]] = {}
current_difficulty_state: Dict[str, str] = {}

def start_mcq_session(student_id: str, topic: str) -> Dict[str, Any]:
    if student_id not in students:
        return {"error": "Please register/login first in the Student Profile tab."}

    difficulty = current_difficulty_state.get(student_id, "easy")
    mcq = generate_mcq(topic, difficulty)
    current_mcq[student_id] = mcq
    students[student_id]["current_topic"] = topic
    return mcq

def grade_mcq_answer(student_id: str, selected_option: str) -> str:
    if student_id not in current_mcq:
        return "No active question. Please start a quiz first."

    mcq = current_mcq[student_id]
    correct = (selected_option == mcq["answer"])
    feedback = "Correct! 🎉" if correct else f"Incorrect. The correct answer is {mcq['answer']}."

    qa_record = {
        "topic": mcq["topic"],
        "question": mcq["question"],
        "selected": selected_option,
        "correct_answer": mcq["answer"],
        "correct": correct,
        "difficulty": mcq["difficulty"],
        "timestamp": time.time(),
    }
    update_student_progress(student_id, qa_record)

    # Adjust difficulty
    new_diff = adjust_difficulty(students[student_id], correct, mcq["difficulty"])
    current_difficulty_state[student_id] = new_diff

    explanation = mcq.get("explanation", "No explanation available.")
    return f"""{feedback}

Explanation: {explanation}

Next question will be at '{new_diff}' difficulty based on your performance.
"""

In [23]:
def ui_register_student(student_id: str, name: str, level: str):
    if not student_id.strip() or not name.strip():
        return "Student ID and Name are required."
    student = init_student(student_id.strip(), name.strip(), level)
    return f"Profile loaded for {student['name']} (Level: {student['level']}). You can now use all other tabs."

def ui_get_learning_path(student_id: str, name: str, level: str):
    if not student_id.strip() or not name.strip():
        return "Please provide Student ID and Name."
    return personalize_learning_path(student_id.strip(), name.strip(), level)

def ui_start_quiz(student_id: str, topic: str):
    student_id = student_id.strip()
    if not student_id:
        return "Student ID required.", "", "", "", "", ""
    result = start_mcq_session(student_id, topic)
    if "error" in result:
        return result["error"], "", "", "", "", ""
    q = result["question"]
    A = result["options"].get("A", "")
    B = result["options"].get("B", "")
    C = result["options"].get("C", "")
    D = result["options"].get("D", "")
    info = f"Topic: {result['topic']} | Difficulty: {result['difficulty']}"
    return info, q, A, B, C, D

def ui_submit_answer(student_id: str, selected_option: str):
    student_id = student_id.strip()
    if not student_id:
        return "Student ID required."
    if not selected_option:
        return "Please select an option (A/B/C/D)."
    return grade_mcq_answer(student_id, selected_option)

def ui_explain_topic(student_id: str, topic: str, level: str):
    if not topic.strip():
        return "Please select or enter a topic."
    return explain_topic(topic.strip(), level)

def ui_show_progress(student_id: str):
    student_id = student_id.strip()
    if not student_id:
        return "Student ID required."
    if student_id not in students:
        return "No profile found. Please register first."
    # Refresh recommendations
    recommend_next_topics(students[student_id])
    return format_progress_report(student_id)

def ui_chat_with_mentor(student_id: str, message: str):
    student_id = student_id.strip()
    if not message.strip():
        return "Please enter a question or message."
    if student_id not in students:
        # Create a temporary generic profile
        init_student(student_id or "guest", "Guest", "beginner")
    return chatbot_mentor(student_id or "guest", message)

In [24]:
with gr.Blocks(title="AI-Powered Personalized Learning Mentor") as demo:
    gr.Markdown(
        """
# 🎓 AI‑Powered Personalized Learning Mentor with Adaptive Assessments
**Features:** Personalized learning path, adaptive MCQs, difficulty adjustment, student tracking, topic explanations, progress dashboard, recommendations, and chatbot mentor.
"""
    )

    with gr.Tab("1️⃣ Student Profile & Learning Path"):
        gr.Markdown("### Register / Load Student Profile")
        with gr.Row():
            student_id_reg = gr.Textbox(label="Student ID", placeholder="e.g., stu_101")
            name_reg = gr.Textbox(label="Name", placeholder="Your name")
        level_reg = gr.Radio(
            choices=["beginner", "intermediate", "advanced"],
            value="beginner",
            label="Level"
        )
        btn_register = gr.Button("Save / Load Profile")
        reg_output = gr.Textbox(label="Status")

        gr.Markdown("### Personalized Learning Path")
        btn_path = gr.Button("Generate / Update Learning Path")
        path_output = gr.Textbox(label="Learning Path", lines=8)

        btn_register.click(
            ui_register_student,
            inputs=[student_id_reg, name_reg, level_reg],
            outputs=reg_output
        )

        btn_path.click(
            ui_get_learning_path,
            inputs=[student_id_reg, name_reg, level_reg],
            outputs=path_output
        )

    with gr.Tab("2️⃣ Adaptive MCQ Quiz"):
        gr.Markdown("### Start Adaptive Quiz")
        student_id_quiz = gr.Textbox(label="Student ID (same as Profile)")
        topic_quiz = gr.Dropdown(choices=TOPICS, label="Select Topic")
        btn_start_quiz = gr.Button("Start / Next Question")

        quiz_info = gr.Textbox(label="Question Meta", interactive=False)
        quiz_question = gr.Textbox(label="Question", lines=4, interactive=False)
        with gr.Row():
            opt_A = gr.Textbox(label="A", interactive=False)
            opt_B = gr.Textbox(label="B", interactive=False)
        with gr.Row():
            opt_C = gr.Textbox(label="C", interactive=False)
            opt_D = gr.Textbox(label="D", interactive=False)

        selected_option = gr.Radio(
            choices=["A", "B", "C", "D"],
            label="Your Answer"
        )
        btn_submit = gr.Button("Submit Answer")
        feedback_output = gr.Textbox(label="Feedback", lines=6)

        btn_start_quiz.click(
            ui_start_quiz,
            inputs=[student_id_quiz, topic_quiz],
            outputs=[quiz_info, quiz_question, opt_A, opt_B, opt_C, opt_D]
        )

        btn_submit.click(
            ui_submit_answer,
            inputs=[student_id_quiz, selected_option],
            outputs=feedback_output
        )

    with gr.Tab("3️⃣ Topic Explanations"):
        gr.Markdown("### Get Topic-wise Explanation")
        student_id_explain = gr.Textbox(label="Student ID (optional)")
        topic_explain = gr.Dropdown(
            choices=TOPICS + ["Custom Topic"],
            label="Select Topic"
        )
        custom_topic = gr.Textbox(
            label="If 'Custom Topic', type here",
            placeholder="e.g., Pointers in C, Photosynthesis, etc."
        )
        level_explain = gr.Radio(
            choices=["beginner", "intermediate", "advanced"],
            value="beginner",
            label="Explanation Level"
        )
        btn_explain = gr.Button("Explain Topic")
        explanation_output = gr.Textbox(label="Explanation", lines=10)

        def resolve_topic(selected, custom):
            if selected == "Custom Topic":
                return custom
            return selected

        btn_explain.click(
            lambda sid, sel, custom, lvl: ui_explain_topic(sid, resolve_topic(sel, custom), lvl),
            inputs=[student_id_explain, topic_explain, custom_topic, level_explain],
            outputs=explanation_output
        )

    with gr.Tab("4️⃣ Progress Dashboard & Recommendations"):
        gr.Markdown("### View Your Progress and Recommendations")
        student_id_progress = gr.Textbox(label="Student ID")
        btn_progress = gr.Button("Show Progress")
        progress_output = gr.Textbox(label="Progress & Recommendations", lines=15)

        btn_progress.click(
            ui_show_progress,
            inputs=student_id_progress,
            outputs=progress_output
        )

    with gr.Tab("5️⃣ Chatbot Mentor"):
        gr.Markdown("### Ask Your AI Mentor Anything")
        student_id_chat = gr.Textbox(label="Student ID (optional, for personalization)")
        chat_input = gr.Textbox(label="Your Question / Doubt")
        btn_chat = gr.Button("Ask Mentor")
        chat_output = gr.Textbox(label="Mentor Reply", lines=10)

        btn_chat.click(
            ui_chat_with_mentor,
            inputs=[student_id_chat, chat_input],
            outputs=chat_output
        )

demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://271e3785fd832ca910.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
